파이썬 머신러닝 완벽 가이드 ch8. 7 pg. 516~528

---

## 07 문서 군집화 소개와 실습(Opinion Review 데이터 세트)

### 문서 군집화 개념

비슷한 텍스트 구성의 문서를 군집화 하는 것

같은 카테고리 소속으로 분류할 수 있으므루 텍스트 분류 기반의 문서 분류와유사
- 텍스트 분류 기반의 문서 분류 : 사전에 결정 카테고리 값을 가진 학습 데이터 필요
- 문서 군집화 : 필요 X, 비지도학습

### Opinion Review 데이터 세트를 이용한 문서 군집화 수행하기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import glob, os

path = r'/content/drive/MyDrive/ESAA/24-1/자료/OpinosisDataset1.0/topics'
# path로 지정한 디렉터리 밑에 있는 모든 .data 파일의 파일명을 리스트로 취합.
all_files=glob.glob(os.path.join(path, '*.data'))
filename_list=[]
opinion_text=[]

# 개별 파일의 파일명은 filename_list로 취합,
# 개별 파일의 파일 내용은 DataFrame 로딩 후 다시 string으로 변환해 opinion_text list로 취합
for file_ in all_files:
  # 개별 파일을 읽어서 DataFrame으로 생성
  df=pd.read_table(file_, index_col=None, header=0, encoding='latin1')

  # 절대 경로로 주어진 파일명을 가공. 리눅스에서 수행할 때는 다음 \를 /로 변경.
  # 맨 마지막 .data 확장자도 제거
  filename_ = file_.split('/')[-1]
  filename = filename_.split('.')[0]

  # 파일명 list와 파일 내용 list에 파일명과 파일 내용을 추가.
  filename_list.append(filename)
  opinion_text.append(df.to_string())

# 파일명 list와 파일 내용 list 객체를 DataFrame으로 생성
document_df=pd.DataFrame({'filename':filename_list, 'opinion_text':opinion_text})
document_df.head()

,filename,opinion_text
0,comfort_toyota_camry_2007,...
1,battery-life_amazon_kindle,...
2,accuracy_garmin_nuvi_255W_gps,...
3,bathroom_bestwestern_hotel_sfo,...
4,features_windows7,...


- 파일 이름만으로 의견(opinion)의 텍스트가 어떠한 제품/서비스에 대한 리뷰인지 알 수 있다.

In [4]:
from nltk.stem import WordNetLemmatizer
import nltk
import string

nltk.download('punkt')
nltk.download('wordnet')

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
lemmar = WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


문서를 TF-IDF 형태로 피처 벡터화

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english',
                             ngram_range=(1,2), min_df=0.05, max_df=0.85)
# opinion_text 칼럼 값으로 피처 벡터화 수행
feature_vect = tfidf_vect.fit_transform(document_df['opinion_text'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(


군집화 : K-평균 적용
- 5개의 중심(Centroid) 기반
- KMeans 수행 후 군집의 Label값과 중심별로 할당된 데이터 세트의 좌표 값 구하기

In [6]:
from sklearn.cluster import KMeans

# 5개 집합으로 군집화 수행.
km_cluster = KMeans(n_clusters=5, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


각 데이터별로 할당된 군집의 레이블을 'cluster_label'칼럼을 추가해 저장
- 각 파일명은 의견 리뷰에 대한 주제

In [7]:
document_df['cluster_label'] = cluster_label
document_df.head()

,filename,opinion_text,cluster_label
0,comfort_toyota_camry_2007,...,2
1,battery-life_amazon_kindle,...,1
2,accuracy_garmin_nuvi_255W_gps,...,0
3,bathroom_bestwestern_hotel_sfo,...,3
4,features_windows7,...,0


`sort_values(by=정렬칼럼명)`을 통해 '정렬칼럼명'으로 데이터 정렬

In [8]:
# cluster_label=0인 데이터 세트 : 차량용 네비게이션에 대한 리뷰로 군집화
document_df[document_df['cluster_label']==0].sort_values(by='filename')

,filename,opinion_text,cluster_label
2,accuracy_garmin_nuvi_255W_gps,...,0
10,directions_garmin_nuvi_255W_gps,...,0
9,display_garmin_nuvi_255W_gps,...,0
4,features_windows7,...,0
32,satellite_garmin_nuvi_255W_gps,...,0
42,screen_garmin_nuvi_255W_gps,...,0
44,speed_garmin_nuvi_255W_gps,...,0
47,speed_windows7,...,0
46,updates_garmin_nuvi_255W_gps,...,0
50,voice_garmin_nuvi_255W_gps,...,0


In [9]:
# cluster_label=1인 데이터 세트 : 포터블 전자기기에 대한 리뷰로 군집화
document_df[document_df['cluster_label']==1].sort_values(by='filename')

,filename,opinion_text,cluster_label
1,battery-life_amazon_kindle,...,1
6,battery-life_ipod_nano_8gb,...,1
7,battery-life_netbook_1005ha,...,1
27,keyboard_netbook_1005ha,...,1
13,performance_netbook_1005ha,...,1
40,screen_ipod_nano_8gb,...,1
31,screen_netbook_1005ha,...,1
28,size_asus_netbook_1005ha,...,1
36,sound_ipod_nano_8gb,headphone jack i got a clear case for it a...,1
45,video_ipod_nano_8gb,...,1


In [10]:
# cluster_label=2인 데이터 세트 : 자동차에 대한 리뷰로 군집화
document_df[document_df['cluster_label']==2].sort_values(by='filename')

,filename,opinion_text,cluster_label
11,comfort_honda_accord_2008,...,2
0,comfort_toyota_camry_2007,...,2
23,gas_mileage_toyota_camry_2007,...,2
16,interior_honda_accord_2008,...,2
12,interior_toyota_camry_2007,...,2
18,mileage_honda_accord_2008,...,2
26,performance_honda_accord_2008,...,2
29,quality_toyota_camry_2007,...,2
34,seats_honda_accord_2008,...,2
43,transmission_toyota_camry_2007,...,2


In [11]:
# cluster_label=3인 데이터 세트 : 대부분 호텔에 대한 리뷰로 군집화
document_df[document_df['cluster_label']==3].sort_values(by='filename')

,filename,opinion_text,cluster_label
3,bathroom_bestwestern_hotel_sfo,...,3
14,food_holiday_inn_london,...,3
20,food_swissotel_chicago,...,3
17,free_bestwestern_hotel_sfo,...,3
25,location_bestwestern_hotel_sfo,...,3
21,location_holiday_inn_london,...,3
15,parking_bestwestern_hotel_sfo,...,3
35,price_holiday_inn_london,...,3
33,room_holiday_inn_london,...,3
37,rooms_bestwestern_hotel_sfo,...,3


In [12]:
# cluster_label=4인 데이터 세트 : 킨들 대한 리뷰로 군집화
document_df[document_df['cluster_label']==4].sort_values(by='filename')

,filename,opinion_text,cluster_label
8,buttons_amazon_kindle,...,4
5,eyesight-issues_amazon_kindle,...,4
22,fonts_amazon_kindle,...,4
24,navigation_amazon_kindle,...,4
19,price_amazon_kindle,...,4


중심 개수를 5개에서 3개로 낮춰서 3개 그룹으로 군집화

In [13]:
# 3개의 집합으로 군집화
km_cluster = KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

# 소속 군집을 cluster_label 칼럼으로 할당하고 cluster_label 값으로 정렬
document_df['cluster_label'] = cluster_label
document_df.sort_values(by='cluster_label')

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,filename,opinion_text,cluster_label
0,comfort_toyota_camry_2007,...,0
23,gas_mileage_toyota_camry_2007,...,0
43,transmission_toyota_camry_2007,...,0
16,interior_honda_accord_2008,...,0
26,performance_honda_accord_2008,...,0
29,quality_toyota_camry_2007,...,0
12,interior_toyota_camry_2007,...,0
34,seats_honda_accord_2008,...,0
18,mileage_honda_accord_2008,...,0
11,comfort_honda_accord_2008,...,0


- 0: 자동차
- 1: 호텔
- 2: 전자기기


### 군집별 핵심 단어 추출하기

각 군집(Cluster)에 속한 문서는 핵심 단어를 주축으로 군집화돼 있을 것이다

`cluster_centers_`
- KMeans 객체는 각 군집을 구성하는 단어 피처가 군집의 중심(Centroid)을 기준으로 얼마나 가깝게 위치해 있는지
- 배열 값으로 제공. 행은 개별 군집, 열은 개별 피처 의미
- 각 배열 내의 값 : 개별 군집 내의 상대 위치를 숫자 값으로 표현한 일종의 좌표 값

In [14]:
cluster_centers = km_cluster.cluster_centers_
print('cluster_centers shape :', cluster_centers.shape)
print(cluster_centers)

cluster_centers shape : (3, 4611)
[[0.         0.00092551 0.         ... 0.         0.         0.        ]
 [0.         0.00099499 0.00174637 ... 0.         0.00183397 0.00144581]
 [0.01005322 0.         0.         ... 0.00706287 0.         0.        ]]


- 군집이 3개 word 피처가 4611개로 구성되었음
- 각 행의 배열 값은 각 군집 내의 2409개의 피처의 위치가 개별 중심과 얼마나 가까운가를 상대값으로 나타낸 것.(0~1, 1:중심과 가까움)

`cluster_centers_` 속성값을 이용해 각 군집별 핵심 단어 찾기
- `ndarray의 argsort()[:, ::-1]` 이용해 배열 내 값이 큰 순으로 정렬된 위치 인덱스 값 반환(핵심 단어 피처의 이름 출력하기 위해)
- 해당 인덱스 이용해 핵심 단어 이름과 그때의 상대 위치 값 추출

In [15]:
# 군집별 top n 핵심 단어, 그 단어의 중심 위치 상댓값, 대상 파일명을 반환함.
def get_cluster_details(cluster_model, cluster_data, feature_names, clusters_num,
                        top_n_features=10):
  cluster_details={}

  # cluster_centers array 값이 큰 순으로 정렬된 인덱스 값을 반환
  # 군집 중심점 별 할당된 word 피처들의 거리값이 큰 순으로 값을 구하기 위함
  centroid_feature_ordered_ind=cluster_model.cluster_centers_.argsort()[:, ::-1]

  # 개별 군집별로 반복하며 핵심 단어, 그 단어의 중심 위치 상댓값, 대상 파일명 입력
  for cluster_num in range(clusters_num):
    # 개별 군집별 정보를 담을 데이터 초기화
    cluster_details[cluster_num]={}
    cluster_details[cluster_num]['cluster']=cluster_num

    # cluster_centers_.argsort()로 구한 인덱스를 이용해 top n 피처 단어를 구함
    top_feature_indexes=centroid_feature_ordered_ind[cluster_num, :top_n_features]
    top_features=[feature_names[ind] for ind in top_feature_indexes]

    # top_feature_indexes를 이용해 해당 피처 단어의 중심 위치 상댓값 구함.
    top_feature_values=cluster_model.cluster_centers_[cluster_num,
                                                      top_feature_indexes].tolist()

    # cluster_details 딕셔너리 객체에 개별 군집별 핵심단어와 중심위치 상댓값, 해당 파일명 입력
    cluster_details[cluster_num]['top_features']=top_features
    cluster_details[cluster_num]['top_features_value']=top_feature_values
    filenames=cluster_data[cluster_data['cluster_label']==cluster_num]['filename']
    filenames=filenames.values.tolist()

    cluster_details[cluster_num]['filenames']=filenames

  return cluster_details

get_cluster_details() 호출 -> cluster_details 반환
- 개별 군집번호, 핵심 단어, 핵심단어 중심 위치 상댓값, 파일명 속성 값 정보
- -> 보기 좋게 표현하고자 print_cluster_details() 함수

In [16]:
def print_cluster_details(cluster_details):
  for cluster_num, cluster_detail in cluster_details.items():
    print('####### Cluster {0}'.format(cluster_num))
    print('Top features:', cluster_detail['top_features'])
    print('Reviews 파일명 :', cluster_detail['filenames'][:7])
    print('===============================================')

get_cluster_details(), print_cluster_details() 호출
- get_cluster_details() 호출 시 인자는 KMeans 군집화 객체, 파일명 추출을 위한 document_df DataFrame, 핵심 단어 추출을 위한 피처명 리스트, 전체 군집 개수, 핵심 단어 추출 개수

In [17]:
feature_names = tfidf_vect.get_feature_names_out()
cluster_details = get_cluster_details(cluster_model = km_cluster, cluster_data = document_df,
                                      feature_names = feature_names, clusters_num = 3, top_n_features = 10)
print_cluster_details(cluster_details)

####### Cluster 0
Top features: ['interior', 'seat', 'mileage', 'comfortable', 'gas', 'gas mileage', 'transmission', 'car', 'performance', 'quality']
Reviews 파일명 : ['comfort_toyota_camry_2007', 'comfort_honda_accord_2008', 'interior_toyota_camry_2007', 'interior_honda_accord_2008', 'mileage_honda_accord_2008', 'gas_mileage_toyota_camry_2007', 'performance_honda_accord_2008']
####### Cluster 1
Top features: ['room', 'hotel', 'service', 'staff', 'food', 'location', 'bathroom', 'clean', 'price', 'parking']
Reviews 파일명 : ['bathroom_bestwestern_hotel_sfo', 'food_holiday_inn_london', 'parking_bestwestern_hotel_sfo', 'free_bestwestern_hotel_sfo', 'food_swissotel_chicago', 'location_holiday_inn_london', 'location_bestwestern_hotel_sfo']
####### Cluster 2
Top features: ['screen', 'battery', 'keyboard', 'battery life', 'life', 'kindle', 'direction', 'video', 'size', 'voice']
Reviews 파일명 : ['battery-life_amazon_kindle', 'accuracy_garmin_nuvi_255W_gps', 'features_windows7', 'eyesight-issues_amazon